In [1]:
import time
import pyautogui
from object.database_connection import DatabaseConnection
from object.dofus_agent import Agent
from object.map_state import MapState
from utils_dofus import ACTIONS, get_player_pos, dict_swap_pos_to_array_index, dict_swap_pos_to_dofus, MONSTER, INVO, BOSS, END_TURN, base_x, base_y, step_y, step_x, box_size

In [2]:
database = DatabaseConnection()
agent = Agent(database, False)

In [3]:
def can_play(screen):
    try:
        if pyautogui.locate("data/game_images/end_turn.png", screen) is not None:
            return True
    except pyautogui.ImageNotFoundException:
        return False
    return False

In [4]:
def get_pa(screen):
    try:
        if pyautogui.locate("data/game_images/pa_6.png", screen) is not None:
            return 6
        if pyautogui.locate("data/game_images/pa_3.png", screen) is not None:
            return 3
        if pyautogui.locate("data/game_images/pa_0.png", screen) is not None:
            return 0
    except pyautogui.ImageNotFoundException:
        return 0
    return 0

In [5]:
def get_pm(screen):
    try:
        if pyautogui.locate("data/game_images/pm_3.png", screen) is not None:
            return 3
        if pyautogui.locate("data/game_images/pm_2.png", screen) is not None:
            return 2
        if pyautogui.locate("data/game_images/pm_1.png", screen) is not None:
            return 1
        if pyautogui.locate("data/game_images/pm_0.png", screen) is not None:
            return 0
    except pyautogui.ImageNotFoundException:
        return 0
    return 0

In [6]:
def center_cell(dofus_pos):
    x = base_x + step_x * dofus_pos[0]
    y = base_y + step_y * dofus_pos[1]
    
    if dofus_pos[2]:
        x += step_x // 2
        y += step_y // 2
        
    x += box_size // 2
    y += box_size // 2
    
    return x, y

In [7]:
def do_for_real(state, action):

    clicked = ACTIONS[action]
    player_pos = get_player_pos(map_state.map)

    if player_pos == -1:
        return False
    
    if clicked == END_TURN:
        pyautogui.keyDown("f1")
        time.sleep(0.01)
        pyautogui.keyUp("f1")
        return True

    clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]

    if clicked_pos not in dict_swap_pos_to_array_index.keys():
        return False
    
    clicked_index = dict_swap_pos_to_array_index[clicked_pos]
    
    if state.map[clicked_index] == MONSTER or state.map[clicked_index] == INVO or state.map[clicked_index] == BOSS:
        pyautogui.keyDown("2")
        time.sleep(0.01)
        pyautogui.keyUp("2")
    
    pyautogui.moveTo(center_cell(dict_swap_pos_to_dofus[clicked_pos]))
    pyautogui.click()
    
    return True

In [8]:
while 1:
    time.sleep(1)
    screenshot = pyautogui.screenshot()
    if can_play(screenshot):
        map_state = MapState.screen_info(get_pa(screenshot), get_pm(screenshot), screenshot=screenshot)
        action, _ = agent.best_action(map_state)
        do_for_real(map_state, action)
    else:
        print("Searching for action to do")

Searching for action to do
Searching for action to do


KeyboardInterrupt: 